## Importing all the csv tables
1. df_symphonies - a list of algos with their composer URL
2. df_oos - a list of OOS (out of sample) stats on individual algos
3. df_backtest - a list of **detail** stats based on the longest possible backtest
4. df_oos_stats - a list of **detail** OOS stats on individual algos

In [1]:
import pandas as pd

df_symphonies = pd.read_csv('bin/symphonies-2024-12-30/SYMPHONIES.csv')
df_oos = pd.read_csv('bin/symphonies-2024-12-30/OOS.csv')
df_backtest = pd.read_csv('bin/symphonies-2025-01-11/BACKTEST.csv')
df_oos_stats = pd.read_csv('bin/symphonies-2025-01-11/OOS.csv')

In [2]:
merged_df = pd.merge(df_symphonies, df_oos, on='symphony_sid', how='left')
merged_df = pd.merge(merged_df, df_oos_stats, left_on='symphony_sid', right_on='sid', how='left')

merged_df.shape

(4512, 112)

# Column name checkers on the tables


In [30]:
list(filter(lambda x: 'profit' in x, list(merged_df.columns)))

['profit_factor', 'profit_ratio']

# Filter and sort the Algos based on a certain criteria on the stats
## We use the OOS table rather than backtest table, because backtest data tends to be extremely overfit.
1. Win Days > 51% - prefer algos that wins more than loss
2. CAGR ≤ 1000% - filter out overfit algos
3. Start Period on or before 1/1/2024 - prefer stats with at least >1 year of data
4. End period on or after 12/2024 - some algo may includes tickers that terminates in between, and thus useless.
5. Payoff Ratio ≥ 1.25
6. Profit Factor ≥ 1

In [43]:
def get_date(row, key):
    return '2024-12-28' if isinstance(row[key], float) else row[key][:10]

def extract_start_date(row):
    return pd.to_datetime(get_date(row, 'last_semantic_update_at'))

def extract_last_date(row):
    return pd.to_datetime(get_date(row, 'last_backtest_last_market_day'))

merged_df['backtest_start_date'] = merged_df.apply(extract_start_date, axis=1)
merged_df['backtest_last_date'] = merged_df.apply(extract_last_date, axis=1)

In [49]:
# df_oos.columns
win_rate_gt_50 = merged_df['win_rate'] > .5 
cagr_lt_1000 = merged_df['cagr'] < 10
oos_start_gt = merged_df['backtest_start_date'] < '2024-01-01'
oos_last_day = merged_df['backtest_last_date'] >= '2024-12-29'
payoff_gt = merged_df['payoff_ratio'] >= 1.25
profit_gt = merged_df['profit_factor'] >= 1
filter_cols = ['symphony_sid', 'title', 'smart_sharpe', 'win_rate', 'cagr', 'payoff_ratio', 'profit_factor', 'backtest_start_date']
merged_df[
    win_rate_gt_50 & cagr_lt_1000 & oos_start_gt & oos_last_day & payoff_gt & profit_gt
][
    filter_cols
].sort_values(by = ['cagr'], ascending=False).head(50)

,symphony_sid,title,smart_sharpe,win_rate,cagr,payoff_ratio,profit_factor,backtest_start_date
1759,wadbe3IfwvSES5vk6yiu,TESTPORT #019: Top Cap by MA + RSI ETF Hedge |...,2.187080,0.556701,1.886272,1.272242,1.597700,2023-06-13
2400,10gAZFrWUyt0yKglAD6O,Monthly MAR SS · Strategy on Composer,2.044197,0.534884,1.082108,1.288520,1.481798,2023-12-18
1157,4nRRutmlzZpacQYQCrXs,NVDA or V1a Simple Sort · Strategy on Composer,2.027890,0.537879,0.875719,1.307850,1.522252,2023-12-09
2306,mPEwXoPyph4KDiTqIY97,V1b A Better,2.154799,0.547794,0.858497,1.257019,1.522731,2023-11-28
3592,oMx2o5HqFfQSa9n1Se8i,V1a Simple Portfolio (UVXY) + BB V3.0.4.2A mer...,2.231058,0.577830,0.827675,1.253758,1.716037,2023-04-24
4293,vLfcx19DP0jTKjVSt4da,V1a Simple Portfolio (UVXY) + BB V3.0.4.2A mer...,2.231058,0.577830,0.827675,1.253758,1.716037,2023-04-23
1331,Kak9fBeaxnTfhM7oBSZn,Rage · Strategy on Composer,2.351413,0.583924,0.803142,1.267031,1.778163,2023-04-23
1506,5LyrJXtkTrV1xzxjfqjd,V1a Simple Portfolio (UVXY) + v4 Pops + BB V3....,2.351413,0.583924,0.803142,1.267031,1.778163,2023-04-23
3583,8LNlmTgqsYnOOQ2RXMOo,V1a Simple Portfolio (UVXY) + v4 Pops · Strate...,2.168592,0.568396,0.740002,1.292272,1.701845,2023-04-22
558,bfDWmOdCVXh1xHw92AdJ,Ease Up on the Gas V2a (add a little nitro) · ...,1.664646,0.576531,0.729495,1.411190,1.921259,2023-06-06
